<font size="3">__Team 3 Final Project: Ships in San Francisco__

Kelli Byrne, Elizabeth Haddad, Runze Wang

November 17, 2021</font>

________
# 1.0 Problem Statement
________


________
# 2.0 Dataset
________


We will using the ‘Objects 2d: Ships in San Francisco’ dataset available here: https://www.kaggle.com/rhammell/ships-in-satellite-imagery

As the Kaggle website explains, the dataset consists of image chips extracted from Planet satellite imagery collected over the San Francisco Bay and San Pedro Bay areas of California. It includes 4000 80x80 RGB images labeled with either a "ship" or "no-ship" classification, and is also distributed as a JSON formatted text file shipsnet.json. The loaded object contains data, label, scene_ids, and location lists.

We have also applied for Planet’s Education and Research Program to gain free access to additional satellite imagery of San Francisco Bay with timestamp data.

We have also downloaded a port boundary polygon from the State of California GIS data, available here: https://gis.data.ca.gov/datasets/ca25a3a15fa2448c801499638300780e_0/explore


## 2.1 Import Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as clrs
import seaborn as sns
import numpy as np

## 2.2 Import JSON

In [ ]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#List items in data folder on Google Drive
!ls '/content/drive/MyDrive/EM-0213/shipsnet'

scenes	shipsnet  shipsnet.json


In [ ]:
#Save JSON data as variable
shipsdata = pd.read_csv('/content/drive/MyDrive/EM-0213/shipsnet/shipsnet.json')


## 2.2 Read JSON

In [ ]:
# Read the shipsnet.json
pd.read_json('/content/drive/MyDrive/EM-0213/shipsnet/shipsnet.json')

,data,labels,locations,scene_ids
0,"[82, 89, 91, 87, 89, 87, 86, 86, 86, 86, 84, 8...",1,"[-118.2254694333423, 33.73803725920789]",20180708_180909_0f47
1,"[76, 75, 67, 62, 68, 72, 73, 73, 68, 69, 69, 6...",1,"[-122.33222866289329, 37.7491755586813]",20170705_180816_103e
2,"[125, 127, 129, 130, 126, 125, 129, 133, 132, ...",1,"[-118.14283073363218, 33.736016066914175]",20180712_211331_0f06
3,"[102, 99, 113, 106, 96, 102, 105, 105, 103, 10...",1,"[-122.34784341495181, 37.76648707436548]",20170609_180756_103a
4,"[78, 76, 74, 78, 79, 79, 79, 82, 86, 85, 83, 8...",1,"[-122.34852408322172, 37.75878462398653]",20170515_180653_1007
...,...,...,...,...
3995,"[126, 122, 124, 138, 165, 186, 195, 199, 203, ...",0,"[-122.08693255500694, 37.77781408256089]",20170815_180821_102d
3996,"[130, 134, 139, 128, 117, 126, 141, 147, 142, ...",0,"[-122.10549691828378, 37.76946626247702]",20170730_191230_0f21
3997,"[171, 135, 118, 140, 145, 144, 154, 165, 139, ...",0,"[-122.48298739296371, 37.684929808845375]",20161116_180804_0e14
3998,"[85, 90, 94, 95, 94, 92, 93, 96, 93, 94, 94, 9...",0,"[-122.29028216570079, 37.71632091139081]",20170211_181116_0e16


________
# 3.0 Data Cleaning
________


________
# 4.0 Visualizations
________

________
# 5.0 Ethical Considerations
________